In [1]:
# Create a custom reducer

# import helpers to define reducer scope
from learning_observer.stream_analytics.helpers import kvs_pipeline, KeyField, Scope

@kvs_pipeline(scope=Scope([KeyField.STUDENT]), module_override='testing')
async def event_counter(event, state):
    '''This is a simple reducer to count the total
    events for a given scope.
    '''
    if state is None:
        state = {}
    state['event_count'] = state.get('event_count', 0) - 1
    return state, state

In [7]:
# Implement reducer into system with our h
ID = 'event_counter'
module = 'example_mod'

import learning_observer.interactive_development
reducer = learning_observer.interactive_development.construct_reducer(ID, event_counter, module=module, default={'event_count': 0})
await learning_observer.interactive_development.hot_load_reducer(reducer, reload=True, migration_function=learning_observer.interactive_development.DROP_DATA)

In [3]:
import learning_observer.module_loader 
print(learning_observer.module_loader.reducers())

[{'context': 'org.mitros.writing_analytics', 'function': <function time_on_task at 0x7f88368b1e10>, 'scope': Scope({<EventField.doc_id>, <KeyField.STUDENT: 1>}), 'default': {'saved_ts': 0}, 'module': <module 'writing_observer.module' from '/home/brad/github/writing_observer/modules/writing_observer/writing_observer/module.py'>, 'id': 'writing_observer.time_on_task'}, {'context': 'org.mitros.writing_analytics', 'function': <function reconstruct at 0x7f88368b1f30>, 'scope': Scope({<EventField.doc_id>, <KeyField.STUDENT: 1>}), 'default': {'text': ''}, 'module': <module 'writing_observer.module' from '/home/brad/github/writing_observer/modules/writing_observer/writing_observer/module.py'>, 'id': 'writing_observer.reconstruct'}, {'context': 'org.mitros.writing_analytics', 'function': <function event_count at 0x7f88368b2050>, 'scope': Scope({<KeyField.STUDENT: 1>}), 'default': {}, 'module': <module 'writing_observer.module' from '/home/brad/github/writing_observer/modules/writing_observer/wr

In [9]:
import learning_observer.kvs

kvs = learning_observer.kvs.KVS()
len(await kvs.keys())

211

In [10]:
# Create a dashboard to connect to the reducer you just wrote
# This dashboard creates a graph for "Total events over time"
import dash
from dash import Dash, html, dcc, callback, Output, Input, State, clientside_callback, Patch
import time
import json
import lo_dash_react_components as lodrc
import pandas as pd
import plotly.graph_objects as go

app = Dash(__name__)

fig = go.Figure(data=go.Scatter(
    x=pd.Series(dtype=object), y=pd.Series(dtype=object)
))

# create app layout
app.layout = html.Div([
    html.H4('Graph of event count'),
    dcc.Graph(id='graph', figure=fig),
    html.H4('Incoming data.'),
    lodrc.LOConnection(id='ws', url='ws://localhost:9999/wsapi/communication_protocol')
])

# Receive message from websocket and update graph
clientside_callback(
    '''function(msg) {
        if (!msg) {
            return window.dash_clientside.no_update;
        }
        // extract data from message
        const data = JSON.parse(msg.data);
        console.log(data);
        const students = data.test.event_count;
        if (students === undefined) { return window.dash_clientside.no_update; }
        if (students.length === 0) {
            return window.dash_clientside.no_update;
        }
        // prep data for dcc.Graph.extendData
        const studentIndex = 0;
        const x = [Date.now() / 1000];
        const y = [students[studentIndex].event_count];
        return [
            { x: [x], y: [y] },
            [0]
        ];
    }''',
    Output('graph', 'extendData'),
    Input('ws', 'message')
)
    
# Send connection information on the websocket when the connectedj
# NOTE that this uses an f''' (triple quote) string.
# Any curly braces need to be doubled up because of this.
clientside_callback(
    f'''function(state) {{
        if (state === undefined) {{
            return window.dash_clientside.no_update;
        }}
        if (state.readyState === 1) {{
            return JSON.stringify({{"test": {{"execution_dag": "{module}", "target_exports": ["event_count"], "kwargs": {{"course_id": 12345}}}}}});
        }}
    }}''',
    Output('ws', 'send'),
    Input('ws', 'state')
)

# `jupyter_mode='inline'` will run the dashboard below
# `supress_callback_exceptions=True` will prevent dash
# from warning you about callbacks with missing IDS.
# These callbacks are from other dashboards.
app.run_server(jupyter_mode='inline', suppress_callback_exceptions=True)
